<a href="https://colab.research.google.com/github/Kcurr01/HUT_Research/blob/main/VAE_11_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
Instalation 
---

In [ ]:
!pip install captum
!pip install umap-learn
!pip install datashader
!pip install bokeh
!pip install holoviews

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 5.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 88 kB 3.5 MB/s 
     |████████████████████████████████| 1.1 MB 12.6 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82830 sha256=d51e2806fb08fcd1d0e7d1ceca02851067f6973c36a94b65c961d55907d38f90
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.8-py3-none-any.whl size=55509 sha256=139bf2e13a7cce7415a32351fecc31b3fec09eda8e3f734685a0efd94a4d3db6
  Stored in directory: /root/.cache/pip/wheels/19/bc/eb/974072a56a7082a302f8b4be1ad6d21bf5019235c2eff65928
Successfully built umap-learn pynndescent
Looking in indexes: https://pypi.org/simple, https://us-p

---
VAE Initializaiton, Visualization and Training
--- 

In [ ]:
import torch   
import torch.nn as nn                          
import torch.nn.functional as F                
import torch.optim as optim   
from torch.utils.data import TensorDataset, Dataset, DataLoader
from torch.autograd import Variable
import torch.utils.data

import pandas as pd 
import numpy as np
import os                             

from captum.attr import LayerConductance, LayerActivation, LayerIntegratedGradients
from captum.attr import IntegratedGradients, DeepLift, GradientShap, NoiseTunnel, FeatureAblation

import matplotlib.pyplot as plt
import seaborn as sns    
import plotly.offline as py
import plotly.graph_objs as go        
                
from tqdm import tqdm

import umap
import umap.plot

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#based on sensor data can you determine the stimulus that is currently in use?

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/WEAR_LAB/Research_Pytorch/S1_E1_A1_v3.csv")

In [ ]:
#df = df.drop(columns=['series_id'])
df.head()

In [ ]:
#df.describe()
#df.plot()

In [ ]:
X = df.iloc[:,1:]
y = df.iloc[:, 0:1]
print(X.shape, type(X), y.shape, type(y))
print()
#print(y.value_counts())

In [ ]:
X.head()

In [ ]:
y.head()

---
Visulaize number of lables

In [ ]:
sns.countplot(x = 'stimulus', data=df)
df.loc[:,'stimulus'].value_counts()

In [ ]:
# Grouping
stimulus = df.groupby("stimulus")
stimulus.mean()

---
Visualize Data Disturbutions


In [ ]:
# #distribution of first 16 features


# fig, axs = plt.subplots(nrows=11, ncols=4, figsize=(60, 60))
# axs = axs.flatten()
# index = 0
# for k, v in df.items():
#   print(f"[{index +1}] Updating plot")
#   sns.distplot(v, ax=axs[index])
#   index += 1
#   if index == 43:
#     break 
# plt.tight_layout()


In [ ]:
X = df.iloc[:,1:].values
y = df.iloc[:, 0:1].values
print(X.shape, type(X), y.shape, type(y))

# Data Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42) # 0.25 x 0.8 = 0.2

print(f"X_train size: {len(X_train)} | X_val size: {len(X_val)} | X_test size: {len(X_test)}")
print(f"y_train size: {len(y_train)} | y_val size: {len(y_val)} | y_test size: {len(y_test)}")
print()
print(f"Training Feature Split: {X_train.shape} | Training Labels { y_train.shape}")
print(f"Validation Feature Split: {X_val.shape} | Validation Labels { y_val.shape}")
print(f"Testing Feature Split: {X_test.shape} | Testing Labels { y_test.shape}")
print()

#Normalization Data 
Minmax = preprocessing.MinMaxScaler()
#Standardized = preprocessing.StandardScaler()
X_train_Minmax= Minmax.fit_transform(X_train)
X_val_Minmax = Minmax.transform(X_val)
X_test_Minmax = Minmax.transform(X_test)

#Convert to numpy then to torch 

X_train = torch.from_numpy(X_train_Minmax).float()
y_train = torch.from_numpy(y_train).float()

X_val = torch.from_numpy(X_val_Minmax).float()
y_val = torch.from_numpy(y_val).float()

X_test = torch.from_numpy(X_test_Minmax).float()
y_test = torch.from_numpy(y_test).float()

print(f"X_train: {type(X_train)} | y_train {type(y_train)}")
print(f"X_val: {type(X_val)} | y_train {type(y_val)}")
print(f"X_test: {type(X_test)} | y_test {type(y_test)}")
print()
print(f"Training: {X_train.shape} , { y_train.shape}")
print(f"Validation: {X_val.shape} , { y_val.shape}")
print(f"Testing:  {X_test.shape} , { y_test.shape}")

In [ ]:
class ClassifierDataset(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data

    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]

    def __len__ (self):
        return len(self.X_data)

training = ClassifierDataset(X_train, y_train)
validating = ClassifierDataset(X_val, y_val)
testing = ClassifierDataset(X_test, y_test)

x_sample, y_sample = training[0]
print(f'elements of x_sample : \n{x_sample[:10]} \nand y_sample : \n{y_sample}')
print(f'x_sample shape : {x_sample.shape} and y_sample shape : {y_sample.shape}')

###############################################################################################
#Hyperparameters
latent_dim = 2
input_dim= 41
hidden_dim= 20
output_dim = 41
num_epochs= 20
batch_size= 100
num_classes = 13
learning_rate= 0.0001 #3e-4 #Karpathy constant

#beta = 1
beta = 0.05
alpha = 1

train_loader = DataLoader(training, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(validating, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(testing, batch_size=batch_size, shuffle=False)


In [ ]:
# classification and matrix 

# from sklearn.neighbors import KNeighborsClassifier
# knn = KNeighborsClassifier(n_neighbors = 3)
# knn.fit(X_train,y_train.ravel())
# prediction = knn.predict(X_train)
# print('With KNN (K=3) accuracy is: ',knn.score(X_test,y_test)) # accuracy
# print('Prediction: {}'.format(prediction))
# print("-------------\n")

# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import classification_report, confusion_matrix
# rf = RandomForestClassifier(random_state = 4)
# rf.fit(X_train,y_train.ravel())
# y_pred = rf.predict(X_test)
# cm = confusion_matrix(y_test,y_pred)
# print('Confusion matrix: \n',cm)
# print('Classification report: \n',classification_report(y_test,y_pred))
# y_prediciton = rf.predict(X_test)
# score = accuracy_score(y_test, y_prediciton)
# sns.heatmap(cm,annot=True,fmt="d") 
# plt.show()

In [ ]:
class VAE(nn.Module):  
  def __init__(self, input_dim, hidden_dim, latent_dim):
    super(VAE,self).__init__()  
    self.fc1 = nn.Linear(input_dim, hidden_dim)  # no labels
    self.mu = nn.Linear(hidden_dim, latent_dim)   # mu
    self.logvar = nn.Linear(hidden_dim,latent_dim)   # log-var

    self.fc3 = nn.Linear(latent_dim, hidden_dim) 
    self.fc4 = nn.Linear(hidden_dim, input_dim)
    
    self.classifier = nn.Sequential(
        nn.Linear(latent_dim, 13),
        nn.Sigmoid(),
        nn.Softmax(dim=1)
    )

  def encode(self, x):     
#    print(f'encoder {type(x)}')         
    z = F.relu(self.fc1(x))
    z = torch.tanh(z) 
    z1 = self.mu(z)               
    z2 = self.logvar(z) 
    return z1, z2                 # (mu, log-var)

  def decode(self, x):
#    print(f'decoder {type(x)}')
    z = F.relu(self.fc3(x))                    
    z = torch.sigmoid(self.fc4(z))      # in [0, 1]
    #print(f"z: {z}")
    return z 

  def forward(self, x):
#    print(f'forward {type(x)}')

#  Reparamaterize
    mu, logvar = self.encode(x)
    stdev = torch.exp(0.5 * logvar)
    esp = torch.randn_like(stdev)
    z_reparmeterized = mu + (esp * stdev)   
    #print(f"z_reparmeterized : {z_reparmeterized}")      
    x_reconstructed = self.decode(z_reparmeterized)
    #print(f"x_reconstructed : {x_reconstructed}")

    classified = self.classifier(z_reparmeterized)

    return (x_reconstructed, z_reparmeterized, classified, mu, logvar)

if __name__ == "__main__":
  x = torch.rand(batch_size,input_dim)
  vae = VAE(input_dim, hidden_dim, latent_dim)
  x_reconstructed, z_reparmeterized, classified, mu, logvar = vae(x)
  print(x_reconstructed.shape)
  print(mu.shape)
  print(logvar.shape)
  print(z_reparmeterized.shape)
  print(classified.shape)


In [ ]:
model  = VAE(input_dim, hidden_dim, latent_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

print(model)
loss_fn = nn.MSELoss(reduction="sum")
classifier_loss_fn = nn.CrossEntropyLoss()

def accuracy(y_pred, y_act):
  y_pred = torch.round(y_pred)
  correct = (y_pred == y_act)
  acc1 = correct.sum()/len(correct)
  acc2 = torch.round(acc1*100)
  # print(f"z_pred: {y_pred} | lable: {y_act} | correct: {correct} | accuracy {acc1} | accuracy {acc2}")
  return acc2

In [ ]:
train_losses=[]
val_losses=[]
train_accuracy = []
val_accuracy = []


dic = dict(latent_space = list(), mu_list=list(), logsig2_list=list(), y=list())
for epoch in range(num_epochs):
  model.train()
  #print(f"zvalue loop begin {z}")
  train_running_loss = 0
  train_running_acc = 0
#  loop = tqdm(train_loader)
  for i, data in enumerate(train_loader):
    inputs, labels = data
    #print(f'type data: {type(data)}')
    #print(f'type inputs: {type(inputs)}')
    #print(f'type labels: {type(labels)}')

    x_reconstructed, z_reparmeterized, classified, mu, logvar = model(inputs)
    #print(x_reconstructed, type(x_reconstructed))
    #print(mu, type(mu))
    #print(logvar, type(logvar))

    reconstruction_loss = loss_fn(x_reconstructed, inputs)
    kld_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    loss = alpha*reconstruction_loss + kld_loss*beta

    acc_train = accuracy(classified, labels)
    #print(reconstruction_loss, kld_loss, loss)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    train_running_loss += loss.item()
    train_loss= train_running_loss/len(train_loader)

    train_running_acc += acc_train.item()
    train_acc = train_running_acc/len(train_loader)

  
  z_list, means, logvars , labels = list(), list(), list(), list()

  #Evaluation
  with torch.inference_mode():
    val_running_loss = 0

    model.eval()
    for X, Y in val_loader:
      #print(labels)
      #inputs = torch.autograd.Variable(inputs)
      y_pred, z_reparmeterized, classified, mu, logvar = model(X)
      v_reconstruction_loss = loss_fn(y_pred, X)
      v_kld_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
      vloss = alpha*v_reconstruction_loss + v_kld_loss*beta
      #print(v_reconstruction_loss, v_kld_loss, vloss)

      # yhat = torch.max(z.data,1)
      # correct+=(yhat==y_test).sum().int()
      # accuracy = correct / n_test
      # accuracy_list.append(accuracy)
  

      # val_acc = accuracy(classified, labels)
      val_acc = 0

      val_running_loss += vloss.item()
      val_loss = val_running_loss/len(val_loader)

      # log ...
      z_list.append(z_reparmeterized.detach())
      means.append(mu.detach())
      logvars.append(logvar.detach())
      labels.append(Y.detach())

  dic['latent_space'].append(torch.cat(z_list))
  dic['mu_list'].append(torch.cat(means))
  dic['logsig2_list'].append(torch.cat(logvars))
  dic['y'].append(torch.cat(labels))

  print(f"Epoch: {epoch+1} / {num_epochs} | reconst_loss: {reconstruction_loss:.3f} | kldiv loss: {kld_loss:.5f} | total loss: {train_loss:.3f} | train acc: {train_acc:.3f} ||| Val Loss: {val_loss:.3f} | val acc: {val_acc:.3f}")
  print("------------------------------------------------------------------------------------------------------------------")
  #print(f"Epoch: {epoch+1} / {num_epochs} | reconst_loss: {v_reconstruction_loss:.3f} | kldiv loss: {v_kld_loss:.3f} | Val Loss: {val_loss:.3f}")
  train_losses.append(train_loss)
  val_losses.append(val_loss)
  train_accuracy.append(train_acc)
  



In [ ]:
torch.save(model.state_dict(), 'VAE_Model.pt') # Save

In [ ]:
plt.plot(train_losses,'-o', label="Training loss")
plt.plot(val_losses,'-r',  label="Validation loss")
plt.xlabel('epoch')
plt.ylabel('losses')
plt.title('Training and Validation Losses')
plt.legend()
plt.show()

In [ ]:
plt.plot(train_accuracy,'-o', label="Training acc")
# plt.plot(val_acc,'-r',  label="Validation acc")
plt.xlabel('epoch')
plt.ylabel('acc')
plt.title('Training and Validation Losses')
plt.legend()
plt.show()

In [ ]:
print("Latent Space Visualization")
for i in range (num_epochs):
  fig = plt.figure(figsize=(10,10))
  z_arr = dic['latent_space'][i].cpu().numpy()
  y_arr = dic['y'][i].cpu().numpy()

  #Experiment 1
  plt.scatter(z_arr[:,0], z_arr[:,1], c = y_arr, edgecolor='none', alpha=0.5,
              cmap=plt.cm.get_cmap('hsv', 13))
  cb = plt.colorbar(ticks=[0,1,2,3,4,5,6,7,8,9,10,11,12],values=[0,1,2,3,4,5,6,7,8,9,10,11,12])
  #Experiment 3
  # plt.scatter(z_arr[:,0], z_arr[:,1], c = y_arr, edgecolor='none', alpha=0.5,
  #             cmap=plt.cm.get_cmap('hsv', 23))
  #cb = plt.colorbar(ticks=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23],values=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23])
  cb.ax.tick_params(labelsize=10)
  plt.xticks(fontsize= 10)
  plt.yticks(fontsize= 10)
  plt.xlabel('z[0]', fontsize= 10)
  plt.ylabel('z[1]', fontsize= 10)
  plt.title(f'VAE train dataset with latent space Dim=2  Epoch number: {i+1} ', fontsize= 12)
  # plt.show()
  plt.close()
  fig.savefig(f"/content/drive/MyDrive/WEAR_LAB/Research_Pytorch/VAE_Images/VAEtrain_images{i:001}" + ".png")
  print(f"Latent Space Image {i+1} stored.")

import imageio
gif = []
for i in range(num_epochs):
  each_image = imageio.imread(f"/content/drive/MyDrive/WEAR_LAB/Research_Pytorch/VAE_Images/VAEtrain_images{i}" + ".png")# here read all images
  gif.append(each_image)
imageio.mimsave("/content/result.gif",gif)

from IPython.display import Image

fname = '/content/result.gif'
Image(open(fname, 'rb').read())  # local

In [ ]:
# for i in range (num_epochs):
#   z_arr = dic['latent_space'][i].cpu().numpy()
#   y_arr = dic['y'][i].cpu().numpy()
#   plt.figure(figsize = (10,5))
#   plt.subplot(1,2,1)
#   plt.scatter(z_arr[:,0], z_arr[:,1], c = y_arr)
#   plt.colorbar()
#   plt.tight_layout()

---
Classificaiton and Transfer
---

In [ ]:
#Classifiers and PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

rand_seed = 7

test = DecisionTreeClassifier(random_state=rand_seed)
test.fit(X_train, y_train)

y_prediciton = test.predict(X_test)
score = accuracy_score(y_test, y_prediciton)
print(f"Accuracy of model: {score:.4f} ")

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_train, y_train.ravel())

y_knn_pred = knn.predict(X_test)
knn_score = accuracy_score(y_test, y_knn_pred)
print("Accuracy of KNN model : {:.4f}".format(knn_score))


from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
pca = PCA(n_components=2)  # same size as autoencoder latent space
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)
rf = RandomForestClassifier(n_estimators=500, n_jobs=-1).fit(X_train_pca,  y_train.ravel())
print(f'PCA Train Accuracy: {rf.score(X_train_pca, y_train)*100}%')
print(f'PCA Test Accuracy: {rf.score(X_test_pca, y_test)*100}%')

Accuracy of model: 0.9689 
Accuracy of KNN model : 0.9013
PCA Train Accuracy: 100.0%
PCA Test Accuracy: 90.5952380952381%


In [ ]:
y_prediciton

---
Test
---

In [ ]:
# # working VAE with latent space repersentation

# def train_model(beta, epochs, model):
#   dic = dict(latent_space = list(), mu_list=list(), logsig2_list=list(), y=list())
#   for epoch in range(0, epochs + 1):
#     if epoch > 0: 
#       model.train()
#       train_loss =0
#       for X, _ in train_loader:
#         x_reconstructed, z_reparmeterized, mu, logvar = model(X)

#         reconstruction_loss = loss_fn(x_reconstructed, X)
#         kld_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

#         loss = reconstruction_loss + kld_loss*3
#         train_loss += loss.item()
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#       print(f'----> Epoch: {epoch} Average loss: {train_loss / len(train_loader):.4f}')
    
#     # Validation 
#     z_list, means, logvars , labels = list(), list(), list(), list()
#     with torch.no_grad():
#         model.eval()
#         validate_loss = 0
#         for X, Y in test_loader:
#           X = X.to(device)
#           # forward ...
#           x_reconstructed, z_reparmeterized, mu, logvar = model(X)

#           reconstruction_loss = loss_fn(x_reconstructed, X)
#           kld_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
#           validation_loss = reconstruction_loss + kld_loss
#           validate_loss += validation_loss.item()

#           # log ...
#           z_list.append(z_reparmeterized.detach())
#           means.append(mu.detach())
#           logvars.append(logvar.detach())
#           labels.append(Y.detach())
          

#     dic['latent_space'].append(torch.cat(z_list))
#     dic['mu_list'].append(torch.cat(means))
#     dic['logsig2_list'].append(torch.cat(logvars))
#     dic['y'].append(torch.cat(labels))
    
#     print(f'----> Val loss:  {validate_loss / len(val_loader):.4f}')
#   return dic

# dic = train_model(beta, num_epochs, model)

# z_arr = dic['latent_space'][0].cpu().numpy()
# y_arr = dic['y'][0].cpu().numpy()
# plt.figure(figsize = (10,5))
# plt.subplot(1,2,1)
# plt.scatter(z_arr[:,0], z_arr[:,1], c = y_arr)
# #plt.scatter(z_arr[:,1], z_arr[:,2], c = y_arr)
# plt.colorbar()
# plt.tight_layout()

------

In [ ]:
# class Loss_Term(nn.Module):
#   def __init__(self):
#     super(Loss_Term, self).__init__()
#     self.mse_loss = nn.MSELoss(reduction="sum")
#     #self.ce_loss = nn.CrossEntropyLoss(reduction="sum")
# # x_reconstructed is the recon_batch created in the forward in the model
# # x is the original x batch, mu is mu and logvar is logvar
#   def forward(self, x_reconstructed,x, mu, logvar):
#     #loss_CE = self.ce_loss(x_reconstructed, x)
#     loss_MSE = self.mse_loss(x_reconstructed, x)
#     loss_KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
#     return loss_MSE + loss_KLD 
#     #return loss_CE + loss_KLD

# loss_func = Loss_Term()

# val_losses = []
# train_losses = []

# def train(epoch):
#     model.train()
#     train_loss = 0
#     for (inputs, labels) in train_loader:
#         optimizer.zero_grad()
#         recon_batch, mu, logvar = model(inputs)
#         loss = loss_func(recon_batch, mu, logvar)
#         loss.backward()
#         train_loss += loss.item()
#         optimizer.step()
# #        if batch_idx % log_interval == 0:
# #            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
# #                epoch, batch_idx * len(data), len(trainloader.dataset),
# #                       100. * batch_idx / len(trainloader),
# #                       loss.item() / len(data)))
#     if epoch % 200 == 0:        
#         print('====> Epoch: {} Average loss: {:.4f}'.format(
#             epoch, train_loss / len(train_loader.dataset)))
#         train_losses.append(train_loss / len(train_loader.dataset))
# for epoch in range(1, num_epochs + 1):
#     train(epoch)

In [ ]:
# class Encoder(nn.Module):
#     def __init__(self, latent_dims):
#         super(Encoder, self).__init__()
#         self.linear1 = nn.Linear(41, 20)
#         self.linear2 = nn.Linear(20, latent_dims)

#     def forward(self, x):
#         x = torch.flatten(x, start_dim=1)
#         x = F.relu(self.linear1(x))
#         return self.linear2(x)


# class Decoder(nn.Module):
#     def __init__(self, latent_dims):
#         super(Decoder, self).__init__()
#         self.linear1 = nn.Linear(latent_dims, 20)
#         self.linear2 = nn.Linear(20, 41)

#     def forward(self, z):
#         z = F.relu(self.linear1(z))
#         z = torch.sigmoid(self.linear2(z))
#         return z

# class Autoencoder(nn.Module):
#     def __init__(self, latent_dims):
#         super(Autoencoder, self).__init__()
#         self.encoder = Encoder(latent_dims)
#         self.decoder = Decoder(latent_dims)

#     def forward(self, x):
#         z = self.encoder(x)
#         return self.decoder(z)

# losses = []
# train_losses=[]
# def train(autoencoder, data, epochs=num_epochs):
#     optimizer = optim.Adam(params=autoencoder.parameters(), lr=learning_rate)
#     for epoch in range(epochs):
#         running_loss = 0
#         for x, y in data:
#             x = x.to(device) # GPU
#             optimizer.zero_grad()
#             x_hat = autoencoder(x)
#             loss = ((x - x_hat)**2).sum()
#             loss.backward()
#             optimizer.step()

#             running_loss += loss.item()
#             train_loss=running_loss/len(train_loader)
#             i= 0
#         print(f"Epoch: {epoch+1} / {epochs} | total loss: {(loss)}")
#         train_losses.append(train_loss)
        
#         plt.plot(train_losses,'-o')
#         plt.xlabel('epoch')
#         plt.ylabel('losses')
#         plt.title('Train Losses')
#         plt.show()  

#         # if i % 200 == 0:
#         #       print(epoch)


#     return autoencoder

# latent_dims = 2
# autoencoder = Autoencoder(latent_dims).to(device) # GPU

# data = train_loader

# autoencoder = train(autoencoder, data)

In [ ]:
# def plot_latent(autoencoder, data, num_batches=100):
#     for i, (x, y) in enumerate(data):
#         z = autoencoder.encoder(x.to(device))
#         z = z.to('cpu').detach().numpy()
#         plt.scatter(z[:, 0], z[:, 1], c=y, cmap='tab10')
#         if i > num_batches:
#             plt.colorbar()
#             break
# plot_latent(autoencoder, data)

---
Example VAE
---

In [ ]:
# import torch
# import torchvision
# from torch import nn
# from torch.utils.data import DataLoader
# from torchvision import transforms
# from torchvision.datasets import MNIST
# from matplotlib import pyplot as plt
# # Displaying routine

# def display_images(in_, out, n=1, label=None, count=False):
#     for N in range(n):
#         if in_ is not None:
#             in_pic = in_.data.cpu().view(-1, 28, 28)
#             plt.figure(figsize=(18, 4))
#             plt.suptitle(label + ' – real test data / reconstructions', color='w', fontsize=16)
#             for i in range(4):
#                 plt.subplot(1,4,i+1)
#                 plt.imshow(in_pic[i+4*N])
#                 plt.axis('off')
#         out_pic = out.data.cpu().view(-1, 28, 28)
#         plt.figure(figsize=(18, 6))
#         for i in range(4):
#             plt.subplot(1,4,i+1)
#             plt.imshow(out_pic[i+4*N])
#             plt.axis('off')
#             if count: plt.title(str(4 * N + i), color='w')
# # Set random seeds

# torch.manual_seed(1)
# torch.cuda.manual_seed(1)
# # Define data loading step

# batch_size = 256

# kwargs = {'num_workers': 1, 'pin_memory': True}
# train_loader = torch.utils.data.DataLoader(
#     MNIST('./data', train=True, download=True,
#                    transform=transforms.ToTensor()),
#     batch_size=batch_size, shuffle=True, **kwargs)
# test_loader = torch.utils.data.DataLoader(
#     MNIST('./data', train=False, transform=transforms.ToTensor()),
#     batch_size=batch_size, shuffle=True, **kwargs)
# # Defining the device

# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# # Defining the model

# d = 20

# class VAE(nn.Module):
#     def __init__(self):
#         super().__init__()

#         self.encoder = nn.Sequential(
#             nn.Linear(784, d ** 2),
#             nn.ReLU(),
#             nn.Linear(d ** 2, d * 2)
#         )

#         self.decoder = nn.Sequential(
#             nn.Linear(d, d ** 2),
#             nn.ReLU(),
#             nn.Linear(d ** 2, 784),
#             nn.Sigmoid(),
#         )

#     def reparameterise(self, mu, logvar):
#         if self.training:
#             std = logvar.mul(0.5).exp_()
#             eps = std.new_empty(std.size()).normal_()
#             return eps.mul_(std).add_(mu)
#         else:
#             return mu

#     def forward(self, x):
#         mu_logvar = self.encoder(x.view(-1, 784)).view(-1, 2, d)
#         mu = mu_logvar[:, 0, :]
#         logvar = mu_logvar[:, 1, :]
#         z = self.reparameterise(mu, logvar)
#         return self.decoder(z), mu, logvar

# model = VAE().to(device)
# # Setting the optimiser

# learning_rate = 1e-3

# optimizer = torch.optim.Adam(
#     model.parameters(),
#     lr=learning_rate,
# )
# # Reconstruction + β * KL divergence losses summed over all elements and batch

# def loss_function(x_hat, x, mu, logvar, β=1):
#     BCE = nn.functional.binary_cross_entropy(
#         x_hat, x.view(-1, 784), reduction='sum'
#     )
#     KLD = 0.5 * torch.sum(logvar.exp() - logvar - 1 + mu.pow(2))

#     return BCE + β * KLD
# # Training and testing the VAE

# epochs = 10
# codes = dict(μ=list(), logσ2=list(), y=list())
# for epoch in range(0, epochs + 1):
#     # Training
#     if epoch > 0:  # test untrained net first
#         model.train()
#         train_loss = 0
#         for x, _ in train_loader:
#             x = x.to(device)
#             # ===================forward=====================
#             x_hat, mu, logvar = model(x)
#             loss = loss_function(x_hat, x, mu, logvar)
#             train_loss += loss.item()
#             # ===================backward====================
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()
#         # ===================log========================
#         print(f'====> Epoch: {epoch} Average loss: {train_loss / len(train_loader.dataset):.4f}')
    
#     # Testing
    
#     means, logvars, labels = list(), list(), list()
#     with torch.no_grad():
#         model.eval()
#         test_loss = 0
#         for x, y in test_loader:
#             x = x.to(device)
#             # ===================forward=====================
#             x_hat, mu, logvar = model(x)
#             test_loss += loss_function(x_hat, x, mu, logvar).item()
#             # =====================log=======================
#             means.append(mu.detach())
#             logvars.append(logvar.detach())
#             labels.append(y.detach())
#     # ===================log========================
#     codes['μ'].append(torch.cat(means))
#     codes['logσ2'].append(torch.cat(logvars))
#     codes['y'].append(torch.cat(labels))
#     test_loss /= len(test_loader.dataset)
#     print(f'====> Test set loss: {test_loss:.4f}')
#     display_images(x, x_hat, 1, f'Epoch {epoch}')
# # Generating a few samples

# N = 16
# z = torch.randn((N, d)).to(device)
# sample = model.decoder(z)
# display_images(None, sample, N // 4, count=True)
# # Display last test batch

# display_images(None, x, 4, count=True)
# # Choose starting and ending point for the interpolation -> shows original and reconstructed

# A, B = 1, 14
# sample = model.decoder(torch.stack((mu[A].data, mu[B].data), 0))
# display_images(None, torch.stack(((
#     x[A].data.view(-1),
#     x[B].data.view(-1),
#     sample.data[0],
#     sample.data[1]
# )), 0))
# # Perform an interpolation between input A and B, in N steps

# N = 16
# code = torch.Tensor(N, 20).to(device)
# sample = torch.Tensor(N, 28, 28).to(device)
# for i in range(N):
#     code[i] = i / (N - 1) * mu[B].data + (1 - i / (N - 1) ) * mu[A].data
#     # sample[i] = i / (N - 1) * x[B].data + (1 - i / (N - 1) ) * x[A].data
# sample = model.decoder(code)
# display_images(None, sample, N // 4, count=True)
# import numpy as np
# from sklearn.manifold import TSNE
# from res.plot_lib import set_default
# set_default(figsize=(15, 4))
# X, Y, E = list(), list(), list()  # input, classes, embeddings
# N = 1000  # samples per epoch
# epochs = (0, 5, 10)
# for epoch in epochs:
#     X.append(codes['μ'][epoch][:N])
#     E.append(TSNE(n_components=2).fit_transform(X[-1].detach().cpu()))
#     Y.append(codes['y'][epoch][:N])
# f, a = plt.subplots(ncols=3)
# for i, e in enumerate(epochs):
#     s = a[i].scatter(E[i][:,0], E[i][:,1], c=Y[i], cmap='tab10')
#     a[i].grid(False)
#     a[i].set_title(f'Epoch {e}')
#     a[i].axis('equal')
# f.colorbar(s, ax=a[:], ticks=np.arange(10), boundaries=np.arange(11) - .5)